In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pyngrok
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00


In [3]:
%%writefile app.py
######## Setup ########
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import streamlit as st
import io
import time
from datetime import datetime
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
from tensorflow.keras import layers

#################################################
################ Functions ######################
#################################################

################ Initialize Session State ###############
def initialize_session_state ():
    st.session_state.stage = "00"
    st.session_state.isFirstVisit = ''
    st.session_state.filepathname = ''
    st.session_state.patient = ''
    st.session_state.snellenscore = ''
    st.session_state.snellenscore_prev = ''
    st.session_state.dr_icdr = ''
    st.session_state.dr_icdr_prev = ''
    st.session_state.patientselfevalscore = ''

################ Update session state #################
def set_stage(stage, isFirstVisit = '', filepathname = '', patient = '', snellenscore = '', dr_icdr = '',
              patientselfevalscore = '', snellenscore_prev='', dr_icdr_prev = '',isVisionWorse='', drproghigherscoreprob=''):
    #Stages to control main processing: 00->05->10->20->40
    st.session_state.stage = stage
    #First Visit/New Patient versus Returning Patient
    if isFirstVisit != '':
      st.session_state.isFirstVisit = isFirstVisit
    if filepathname != '':
      st.session_state.filepathname = filepathname
    #Patient Name: Used to identify the patient especially to know if this is a returing patient for progression check (Name matches)
    if patient != '' :
      st.session_state.patient = patient
    #Snellen Eye Chart Score (encoded in a range of 0-100)
    if snellenscore != '':
      st.session_state.snellenscore = snellenscore
    #Snellen Eye Chart Score (encoded in a range of 0-100)  - Previous score to help with progression check
    if snellenscore_prev != '':
      st.session_state.snellenscore_prev = snellenscore_prev
    #Diabetic Retinopathy ICDR-10 Classification 0-4 Score
    if dr_icdr != '':
      st.session_state.dr_icdr = dr_icdr
    #Diabetic Retinopathy ICDR-10 Classification 0-4 Score - Previous score to help with progression check
    if dr_icdr_prev != '':
      st.session_state.dr_icdr_prev = dr_icdr_prev
    #Patient Self Evaluation (Worse, Same, Better) - Previous score to help with progression check
    #if patientselfevalscore != '':
    #  st.session_state.patientselfevalscore = patientselfevalscore
    #Patient Self Evaluation (Worse, Same, Better) - Previous score to help with progression check
    if isVisionWorse != '':
      st.session_state.isVisionWorse = isVisionWorse
    #Diabetic Retinopathy - Sum of higher softmax probabilities to help with progression check
    if drproghigherscoreprob != '':
      st.session_state.drproghigherscoreprob = drproghigherscoreprob

################# Set snellen eye score #################
def set_snelleneyescore(snellenbestline,snellenmissedletters):
  # snellenbestline "Best line" - "20/20", "20/30", "20/40","20/50","20/70","20/100","20/200"
  # snellenmissedletters "Missed letters" - "-1", "-2","-3","-4","-5","-6"
  # Scoring 20/20 => 100. Subtract points for missed letters. e.g. Missing 3 letters => 100-3 = 97
  # Scoring 20/30 => 90. Subtract points for missed letters.
  # Scoring 20/40 => 80. Subtract points for missed letters.
  # Scoring 20/50 => 70. Subtract points for missed letters.
  # Scoring 20/70 => 50. Subtract points for missed letters.
  # Scoring 20/100 => 20. Subtract points for missed letters.
  # Scoring 20/200 => 0. Subtract points for missed letters.

  # Best Line
  if snellenbestline == "20/20":
    score = 100.
  elif snellenbestline == "20/30":
    score = 90.
  elif snellenbestline == "20/40":
    score = 80.
  elif snellenbestline == "20/50":
    score = 70.
  elif snellenbestline == "20/70":
    score = 50.
  elif snellenbestline == "20/100":
    score = 20.
  elif snellenbestline == "20/200":
    score = 0.

  # Missed letters on the line
  if snellenmissedletters != "None":
    score = score + float(snellenmissedletters)
  if score < 0:
    score = 0

  # Save Snellen Score including previous
  if st.session_state.snellenscore_prev == '':
    st.session_state.snellenscore_prev = score
  else:
    st.session_state.snellenscore_prev = st.session_state.snellenscore
  st.session_state.snellenscore = score
  set_stage(stage="10")

################# Function for checking disease #################
def check_for_disease (diseasename,image,savedmodel,prob,pddiseases):
    model = load_model(savedmodel)
    prediction = model.predict(image)
    prediction_prob = prediction [0][0] * 100
    if prediction_prob > prob:
        pddiseases = pddiseases.append({'disease': [diseasename], 'probability': [prediction_prob]}, ignore_index=True)
    return pddiseases

################# Initial Disease Checks #################
def check_for_initial_diseases(input_image):
            # Load previously saved models
            model_dr = '/content/drive/MyDrive/AI/Models/diabetic_retinopathy_model.h5' #Diabetic Retinopathy
            model_glaucoma = '/content/drive/MyDrive/AI/Models/Glaucoma02122024_2.h5' #Glaucoma (increased cup disc)
            model_amd = '/content/drive/MyDrive/AI/Models/amd_model.h5' #AMD
            model_drusens = '/content/drive/MyDrive/AI/Models/drusens_model.h5' #Drusens
            model_hemmorage = '/content/drive/MyDrive/AI/Models/hemorrhage02122024.h5' #Hemmorage
            model_macedema = '/content/drive/MyDrive/AI/Models/macular_edema.h5' #Macular Edema
            model_vasocclusion = '/content/drive/MyDrive/AI/Models/vascular_occlusion.h5' #Vascular Occlusion
            model_nevus = '/content/drive/MyDrive/AI/Models/nevus.h5' #Nevus
            # Check for multiple diseases
            diseases = pd.DataFrame()

            diseases = check_for_disease ('Diabetic Retinopathy',input_image,model_dr,45,diseases)
            diseases = check_for_disease ('Glaucoma',input_image,model_glaucoma,47,diseases)
            diseases = check_for_disease ('AMD',input_image,model_amd,30,diseases)
            diseases = check_for_disease ('Drusens',input_image,model_drusens,30,diseases)
            diseases = check_for_disease ('Hemmorage',input_image,model_hemmorage,30,diseases)
            diseases = check_for_disease ('Macular Edema',input_image,model_macedema,30,diseases)
            diseases = check_for_disease ('Vascular Occlusion',input_image,model_vasocclusion,30,diseases)
            diseases = check_for_disease ('Nevus',input_image,model_nevus,30,diseases)

            # Capture DR classification baseline but do not report
            check_for_disease_progression(input_image)
            if diseases.empty:
              with st.chat_message("user"):
                st.write("Great news. The patient has no diseases!!!")
            else:
              with st.chat_message("user"):
                st.write("The patient has one or more diseases detected. Please return in 3 months for a follow-up")
                for index in diseases.index:
                  # Access row data using the index
                  ind_disease = diseases['disease'][index][0]
                  ind_prob = round(diseases['probability'][index][0],0)
                  st.write(f"{ind_disease} with a probability of {ind_prob}%")

            set_stage(stage="40")

################# Disease Progression - Diabetic retinopathy only #################
def check_for_disease_progression(input_image):
    overall_progression = 'False'
    model_drprog = load_model('/content/drive/MyDrive/AI/Models/bestDRProgPlanE_model.h5')

    #DR_ICDR: International Clinic Diabetic Retinopathy classification with enumerated values from 0 to 4.
      #0 No retinopathy.
      #1 Mild non-proliferative diabetic retinopathy.
      #2 Moderate non-proliferative diabetic retinopathy.
      #3 Severe non-proliferative diabetic retinopathy.
      #4 Proliferative diabetic retinopathy and post-laser status.

    # Make predictions.
    predictions = model_drprog.predict(input_image)  #Will return a list of probabilities for each class (Softmax) in format [[prob for class 0][prob for class 1]....[prob for class 4]]
    predictmax = np.argmax(predictions, axis=1)      #Pick the Class with the highest probability
    predictprob = predictions[0][predictmax] * 100   #..and associated probability of the above class
    predictprob = int(predictprob) #Skip decimals

    # Initialize DR_ICDR previous class for the first time
    if st.session_state.dr_icdr_prev == '':
      st.session_state.dr_icdr_prev = predictmax[0]
    # Set DR_ICDR previous class
    else:
      st.session_state.dr_icdr_prev = st.session_state.dr_icdr
    # Latest DR_ICDR class
    st.session_state.dr_icdr = predictmax[0]

    # Calculate and save probability of a higher score (sum of probabilities for all subsequent higher classes)
    if predictmax == 0:
      st.session_state.drproghigherscoreprob = predictions[0][1] + predictions[0][2] + predictions[0][3] + predictions[0][4]
    elif predictmax == 1:
      st.session_state.drproghigherscoreprob = predictions[0][2] + predictions[0][3] + predictions[0][4]
    elif predictmax == 2:
      st.session_state.drproghigherscoreprob = predictions[0][3] + predictions[0][4]
    elif predictmax == 3:
      st.session_state.drproghigherscoreprob = predictions[0][4]
    elif predictmax == 4:
      st.session_state.drproghigherscoreprob = 0
    st.session_state.drproghigherscoreprob = round(st.session_state.drproghigherscoreprob * 100)

    # ICDR Progression?
    DiseaseProgressionScore_icdr = st.session_state.dr_icdr - st.session_state.dr_icdr_prev
    # Snellen Progression?
    DiseaseProgressionScore_snellen = st.session_state.snellenscore - st.session_state.snellenscore_prev

    ##################################################################################################################################################
    # Remainder not required for new/first visit patient. The above was required even for new patients to capture a baseline of the disease progression
    if st.session_state.isFirstVisit == "Yes":
      return

    #Progression checking algorithm (Set Overall Progression True versus False)
    #AI has classified ICDR to a higher level than previous
    if DiseaseProgressionScore_icdr > 0:
        overall_progression = 'True'
    #Patient snellen eye test - More than 20 points deterioration from before
    if DiseaseProgressionScore_snellen > 20:
        overall_progression = 'True'
    #Patient self evaluation is worse than before and 15% probability of higher ICDR class
    #if st.session_state.patientselfevalscore == "Worse" and st.session_state.drproghigherscoreprob > 15:
    if st.session_state.isVisionWorse == "Worse" and st.session_state.drproghigherscoreprob > 15:
        overall_progression = 'True'

    # Stable. No progression
    if overall_progression == 'False':
        with st.chat_message("user"):
                  st.write("Good news! The disease has not progressed from the last visit")
                  set_stage(stage="40")
    # Disease has progressed
    else:
        with st.chat_message("user"):
                  st.write("The patients eye disease may have progressed from the last visit.")
                  st.write("Schedule a follow up with your opthalmologist within 1 month")
                  st.write("The following criteria was used for this evaluation")
                  st.write("International Clinic Diabetic Retinopathy (ICDR) classification: 0 to 4")
                  st.write("0 => No retinopathy")
                  st.write("1 => Mild non-proliferative")
                  st.write("2 => Moderate non-proliferative")
                  st.write("3 => Severe non-proliferative")
                  st.write("4 => Proliferative")
                  st.write(f"The ICDR score today: {st.session_state.dr_icdr} with a probability of {st.session_state.drproghigherscoreprob} % of a higher ICDR classification")
                  st.write(f"The ICDR score last time: {st.session_state.dr_icdr_prev}")
                  st.write(f"The Snellen Eye Test score today: {st.session_state.snellenscore}")
                  st.write(f"The Snellen Eye Test score last time: {st.session_state.snellenscore_prev}")
                  st.write(f"The patients self assessment of vision compared to the last visit was {st.session_state.isVisionWorse}")
                  set_stage(stage="40")


################# Stage 00: Greeting & set if new versus returning patient #################
def process_stage_00():
  with st.chat_message("user"):
    st.write("Hello 👋")
  patient = st.chat_input("What is the patient's name?")

  # New versus returning patient
  if patient:
    if st.session_state.patient == patient:
      isFirstVisit = "No"
    else:
      isFirstVisit = "Yes"
    set_stage(stage="05",isFirstVisit=isFirstVisit,patient=patient)
    patient = st.chat_input(disabled=True)

################# Stage 05: Snellen Eye test #################
def process_stage_05():
  # New Patient
  if st.session_state.isFirstVisit == "Yes":
    with st.chat_message("user"):
      st.write(f'''Welcome {st.session_state.patient}! We will first check the vision using an Eye Chart
              followed by an examination of an image from the back of the eye.
              We will then review the results and next steps''')
  # Returning Patient
  else:
    with st.chat_message("user"):
      st.write(f'''Welcome back {st.session_state.patient}! We will re-check the vision using an Eye Chart
               and re-examine an image from the back of the eye. We will compare
               with the previous baseline to determine progression of the disease.
               We will then review the results and next steps''')
  time.sleep(2)

  # Snellen Eye test
  with st.chat_message("user"):
    st.write("Lets start by checking the patient's vision")
  time.sleep(2)
  with st.expander("Snellen Eye Chart for Visual Acuity"):
      st.image('/content/drive/MyDrive/AI/Snellen Chart.jpeg')
  st.write("Record the patient's Best Corrected Vision Acuity")
  snellenbestline = st.select_slider("Best line",
                ["20/20", "20/30", "20/40","20/50","20/70","20/100","20/200"])
  snellenmissedletters = st.select_slider("Missed letters",
                ["None", "-1", "-2","-3","-4","-5","-6"])

  st.button('Submit', key='05', on_click=set_snelleneyescore, kwargs={"snellenbestline":snellenbestline,"snellenmissedletters":snellenmissedletters})

################# Stage 10: Returning Patient for follow-up #################
def process_stage_10():
  # Returning Patient
  if st.session_state.isFirstVisit == "No":
    with st.chat_message("user"):
      st.write(f'Thanks {st.session_state.patient} for coming back for a follow up visit!')
    isVisionWorse = st.select_slider("Rate the patient's vision compared to the previous visit",
                  ["Worse", "Same as before", "Better"])
    st.button('Submit', key='2', on_click=set_stage, kwargs={"stage":"20","isVisionWorse":isVisionWorse})
  # New Patient
  elif st.session_state.isFirstVisit == "Yes":
    set_stage(stage="20")

################# Stage 20: Eye photo image upload & Predict/Display Results #################
def process_stage_20():
  # Upload image of the eye
  with st.chat_message("user"):
    st.write("Upload the patient's eye fundus photo")
  #Create a file uploader
  uploaded_file = st.file_uploader("Eye Fundus Photo", type=["jpg", "jpeg"])

  # Check if a file was uploaded
  if uploaded_file is not None:
          # Get the uploaded image
          image_orig = Image.open(uploaded_file)
          image_orig.save(uploaded_file.name)
          with st.spinner(text="OphthoAI is analyzing the image from the back of the eye"):
              time.sleep(2)
              with st.chat_message("user"):
                st.write('''OphthoAI is checking the eye for the following diseases:
                  Age-related macular degeneration(AMD), Glaucoma, Diabetic Retinopathy, Nevus,
                  Macular Edema, Hemmorage, Vascular occlusion, Drusens''')
              with st.chat_message("user"):
                st.write('We appreciate your patience!')
          # Show the original image before downscaling
          with st.chat_message("user"):
              st.write("Here is an image of the patients eye")
              st.image(image_orig, caption='Uploaded Eye Fundus Photo', use_column_width=True)
          # Downscale image to fit training resolution
          image = load_img(uploaded_file.name, target_size=(224, 224))
          plt.imshow(image)

          # Convert image to array, normalize and reshape to match model input shape
          input_image = img_to_array(image)
          input_image = input_image/255 #Normalize to 0-1 range
          input_image = input_image.reshape(1, 224, 224, 3)  # Add batch dimension and match model input shape

          if st.session_state.isFirstVisit == "Yes":  #New patient
            check_for_initial_diseases(input_image)
          else:                                       #Returning patient
            check_for_disease_progression(input_image)

################# Stage 40: Reset #################
def process_stage_40():
  with st.chat_message("user"):
    st.write('Thank you! A copy of the image and report will be sent to the registered Opthalmologist')
  st.button('Restart', key='4', on_click=set_stage, kwargs={"stage":"00"})

#################################################
################ Main Processing ################
#################################################
# Initialize
if 'stage' not in st.session_state:
    initialize_session_state()

# Print title
st.header("OphthoAI")

# Stage 00: Greeting & set if new versus returning patient
if st.session_state.stage == "00":
  process_stage_00()

# Stage 05: Snellen eye test
if st.session_state.stage == "05":
  process_stage_05()

# Stage 10: Returning Patient for follow-up
if st.session_state.stage == "10":
  process_stage_10()

# Stage 20: Eye photo image upload & Predict/Display Results
if st.session_state.stage == "20":
  process_stage_20()

# Stage 40: Reset
if st.session_state.stage == "40":
  process_stage_40()


Writing app.py


In [4]:
from pyngrok import ngrok
!ngrok authtoken 2agZLIWAjmhsZJPHZoOLLTsOion_3EZ8v8EYB5LFLwzx8NjSW

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
def launch_website():
  print ("Click this link to try your web app:")
  if (ngrok.get_tunnels() != None):
    ngrok.kill()
  public_url = ngrok.connect()
  print(public_url)
  !streamlit run --server.port 80 app.py

In [ ]:
launch_website()

Click this link to try your web app:
NgrokTunnel: "https://b136-34-86-111-222.ngrok-free.app" -> "http://localhost:80"


